<a href="https://colab.research.google.com/github/pollinations/pollinations/blob/dev/colabs/pollinator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pollinator

Does the heavy-lifting for **Pollinations.AI** on Google Colab's free GPUs.

Click *Runtime - Run All*, switch back to the **Pollinations** tab of the browser and wait a little for the connection to be established.

![](https://pollinations.ai/Runtime-RunAll.png?)

In [ ]:
#@title Config and Requirements

!pip install --upgrade pip
!pip install papermill json5
from IPython.display import clear_output
from os.path import join, basename
from shlex import quote

from glob import glob
from json5 import loads

from subprocess import Popen, PIPE, STDOUT
from sys import stdout
import os

clear_output()
#ipfs_nodeid = generate_slug(2)#asyncio.run(loop.create_task(get_node_id()))
ipfs_root = "/content/ipfs"
worker_root = "/content"
notebook_path = join(ipfs_root,"input", "notebook.ipynb")

print(f"---Settings---\nIPFS_ROOT: {ipfs_root}")

In [ ]:
#@title Empty and create ipfs root: */content/ipfs*



!rm -r $ipfs_root
!mkdir -p $ipfs_root
!mkdir -p $ipfs_root/input
!mkdir -p $ipfs_root/output

clear_output()

In [ ]:
#@title Install Pollinations CLI

%cd /content
!git clone https://github.com/voodoohop/pollinations.git
%cd /content/pollinations/app
!git pull
!git checkout dev
!git pull
!npm run install_backend
%cd -

clear_output()

In [ ]:
#@title Install and Run IPFS daemon &

#!/content/pollinations/ipfs/ipfsd_install_colab.sh
#!nohup /content/pollinations/ipfs/ipfsd_forever.sh &
#!sleep 5
clear_output()


node_id=!tr -dc A-Za-z0-9 </dev/urandom | head -c 13 ; echo ''

node_id=node_id[0]

print("---Node ID---:", node_id)

In [ ]:
#@title Create connection to frontend UI

import IPython
from google.colab import output

display(IPython.display.Javascript(f'''
  const iframe = document.createElement("iframe")
  iframe.src = `https://pollinations.ai/incolab.html?node={node_id}`
  document.querySelector("#output-area").appendChild(iframe)
  const iframe2 = document.createElement("iframe")
  iframe2.src = `http://localhost:3000/incolab.html?node={node_id}`
  document.querySelector("#output-area").appendChild(iframe2)
'''))




In [ ]:
#@title IPFS state functions

def get_params():
  parameters={}
  for path in glob(f"{ipfs_root}/input/*"):
    key = basename(path)
    if key == "notebook.ipynb":
      continue
    with open(path,'r') as f: 
      value = f.read()
      #print("reading parameter", value)
      try:
        parameters[key] = loads(value)
      except:
        parameters[key] = ""
  return parameters

def get_form_action():
  params = get_params()
  return "formAction" in params and params["formAction"]
  

def wait_submitted(skip_pollinate=False):
  print("Waiting for form submission")
  while not get_form_action() == "submit":
    if skip_pollinate:
      !sleep 2
    else:
      !DEBUG=* pollinate -p $ipfs_root/input --ipns -n $node_id --receive --once > /content/cid

def set_status(status):
  status = "'" + status + "'"
  !echo -n  "$status" > $ipfs_root/output/status
  print("Set status to",status)

def post_content():
  cid_all=!tail -1 /content/cid
  cid = cid_all[0]
  for platform in ["twitter","instagram","telegram","facebook","youtube","linkedin"]:
    social_post_url = f"https://pollinations.ai/.netlify/functions/social-post/{platform}/{cid}"
    !nohup curl $social_post_url &


In [ ]:
#@title Notebook Runner
from time import sleep
import sys
import matplotlib
matplotlib.use('Agg')
output_path = ipfs_root+"/output"


def run():
  print("Running")
  !DEBUG=* pollinate --execute "run_notebook.sh" --ipns -l /content/ipfs/output/log -p $ipfs_root --ipns -n $node_id --debounce 100 > /content/cid
  post_content()

      
while True:
  #!rm -v $output_path/*    
  wait_submitted()
  run()
  !sleep 3


  ipfs/sender Changed files [ { file: 'output/progress_00210.jpg', event: 'add' } ] +5s
  ipfsConnector adding /tmp_8484/output/progress_00210.jpg options {} +5s
  ipfsConnector added QmauSPEuHjquBdx68Vw9qAJdZtrn6bwEjahAff4aeojnQN size Object [AsyncGenerator] {} +63ms
  ipfsConnector Trying to delete /tmp_8484/output/progress_00210.jpg +0ms
  ipfs-http-client:lib:error-handler { Message: 'file does not exist', Code: 0, Type: 'error' } +5s
  ipfsConnector Could not delete. Probably did not exist. +13ms
  ipfsConnector copying to /tmp_8484/output/progress_00210.jpg +0ms
  ipfs/sender publish QmbQo1rmfR2ufpr4AuWGr1avUHyc3uDFBabg8hQs6Q2DLJ +127ms
  ipfsConnector publish pubsub 196gaYIdIRMxI QmbQo1rmfR2ufpr4AuWGr1avUHyc3uDFBabg8hQs6Q2DLJ +51ms
i: 215, loss: 0.711792, losses: 0.0274123, 0.68438

<IPython.core.display.Image object>
  ipfs/sender Changed files [ { file: 'output/progress_00215.jpg', event: 'add' } ] +5s
  ipfsConnector adding /tmp_8484/output/progress_00215.jpg options {} +5s
 